Pytorch-cifar models have 3x3 instead of 7x7 for the first layer. This is significantly better for smaller images??? like cifar. Without this the accuracy is <=90%

Comparing the standard kuangliu models to my rwightman version to training this using lightning (weight-sharing repo)

In [1]:
import os
import random
import pickle
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

from models import *

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [23]:
ws_amt = 8
model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=ws_amt, channel_factor=1,
                         num_classes=10, cifar=True)

for name, p in model.named_parameters():
    print(name, p.size())

conv1.weight torch.Size([64, 3, 3, 3])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([16, 64, 3, 3])
layer2.0.bn1.weight torch.Size([16])
layer2.0.bn1.bias torch.Size([16])
layer2.0.conv2.weight torch.Size([16, 128, 3, 3])
layer2.0.bn2.weight torch.Size([16])
layer2.0.bn2.bias torch.Size([16])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight torch.Siz

In [24]:
model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=1, channel_factor=ws_amt,
                         num_classes=10, cifar=True)

for name, p in model.named_parameters():
    print(name, p.size())

conv1.weight torch.Size([64, 3, 3, 3])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([16, 64, 3, 3])
layer2.0.bn1.weight torch.Size([16])
layer2.0.bn1.bias torch.Size([16])
layer2.0.conv2.weight torch.Size([16, 16, 3, 3])
layer2.0.bn2.weight torch.Size([16])
layer2.0.bn2.bias torch.Size([16])
layer2.0.downsample.0.weight torch.Size([16, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([16])
layer2.0.downsample.1.bias torch.Size([16])
layer2.1.conv1.weight torch.Size([1

In [32]:
ws_amt = 8
ws_model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=ws_amt, channel_factor=1,
                         num_classes=10, cifar=True)

save_path = './saved_models_old/cifar10/resnet18bn_ws'+str(ws_amt)+'_ch1'
checkpoint = torch.load(save_path + '/ckpt.pth')

new_state_dict = {}
for k, v in checkpoint['net'].items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
ws_model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [21]:
# model without redundancy (fraction instead of ws)
small_model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=1, channel_factor=ws_amt,
                         num_classes=10, cifar=True)

In [45]:
small_state_dict = new_state_dict

# sum repeated weights. format [n_out, n_in, k0, k1] -> [n_out, n_in/ws_amt, k0, k1] 
for name, p in ws_model.named_parameters():
    # Skipped the first conv, first block, and the final FC layer
    if (name.split('.')[0] not in ['conv1', 'bn1', 'layer1', 'fc'] and 
          ('layer2.0.conv1' not in name) and ('layer4.1.conv2' not in name)):
            
        # only repeat conv weights (bias is on the output, which is already small)
        if 'conv' in name:
            print('ws model:', name, p.size())
            print('frac model                    :', small_model.state_dict()[name].size())
            wgt_corresp_to_repeat = []
            
            for i in range(ws_amt):
                idxs = torch.LongTensor(list(range(i, p.size()[1], ws_amt)))
#                 print(idxs)
                stuff = torch.index_select(p, dim=1, index=idxs)
                wgt_corresp_to_repeat.append(stuff)
#                 print('stuff.size()', stuff.size())
            wgt_corresp_to_repeat = torch.stack(wgt_corresp_to_repeat, 0)
            print('wgt_corresp_to_repeat.size():  ', wgt_corresp_to_repeat.size())
            weight = wgt_corresp_to_repeat.sum(dim=0)
            print('weight.size():                 ', weight.size())
            print('-------------------------------')
            small_state_dict[name] = weight
            

ws model: layer2.0.conv2.weight torch.Size([16, 128, 3, 3])
frac model                    : torch.Size([16, 16, 3, 3])
wgt_corresp_to_repeat.size():   torch.Size([8, 16, 16, 3, 3])
weight.size():                  torch.Size([16, 16, 3, 3])
-------------------------------
ws model: layer2.1.conv1.weight torch.Size([16, 128, 3, 3])
frac model                    : torch.Size([16, 16, 3, 3])
wgt_corresp_to_repeat.size():   torch.Size([8, 16, 16, 3, 3])
weight.size():                  torch.Size([16, 16, 3, 3])
-------------------------------
ws model: layer2.1.conv2.weight torch.Size([16, 128, 3, 3])
frac model                    : torch.Size([16, 16, 3, 3])
wgt_corresp_to_repeat.size():   torch.Size([8, 16, 16, 3, 3])
weight.size():                  torch.Size([16, 16, 3, 3])
-------------------------------
ws model: layer3.0.conv1.weight torch.Size([32, 128, 3, 3])
frac model                    : torch.Size([32, 16, 3, 3])
wgt_corresp_to_repeat.size():   torch.Size([8, 32, 16, 3, 3])
wei

In [46]:
small_model.load_state_dict(small_state_dict)

RuntimeError: Error(s) in loading state_dict for ResNetLight:
	size mismatch for layer2.0.downsample.0.weight: copying a param with shape torch.Size([128, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([16, 64, 1, 1]).
	size mismatch for layer2.0.downsample.1.weight: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for layer2.0.downsample.1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for layer2.0.downsample.1.running_mean: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for layer2.0.downsample.1.running_var: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for layer3.0.downsample.0.weight: copying a param with shape torch.Size([256, 128, 1, 1]) from chec

In [3]:
# for name, p in model.named_parameters():
#     print(name, p.size())

In [5]:
small_model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=1, channel_factor=ws_amt,
                         num_classes=10, cifar=True)

# save_path = './saved_models_old/cifar10/resnet18bn_ws'+str(ws_amt)+'_ch1'
# checkpoint = torch.load(save_path + '/ckpt.pth')

# state_dict = {}
# for k, v in checkpoint['net'].items():
#     name = k[7:] # remove `module.`
#     state_dict[name] = v
# small_model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for ResNetLight:
	size mismatch for layer2.0.conv2.weight: copying a param with shape torch.Size([64, 128, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
	size mismatch for layer2.0.downsample.0.weight: copying a param with shape torch.Size([128, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 1, 1]).
	size mismatch for layer2.0.downsample.1.weight: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for layer2.0.downsample.1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for layer2.0.downsample.1.running_mean: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for layer2.0.downsample.1.running_var: copying a param with shape torch.Size([128]) from ch

In [9]:
small_state_dict = state_dict

# sum repeated weights. format [n_out, n_in, k0, k1] -> [n_out, n_in/ws_amt, k0, k1] 
for name, p in model.named_parameters():
    # Skipped the first conv, first block, and the final FC layer
    if name.split('.')[0] not in ['conv1', 'bn1', 'layer1', 'fc']:
        print(name, p.size())
        # only repeat conv weights (bias is on the output, which is already small)
        if 'conv' in name:
            print(p.size())            
            wgt_corresp_to_repeat = []
            
            for i in range(ws_amt):
                idxs = torch.LongTensor(list(range(i, p.size()[1], ws_amt)))
                print(idxs)
                stuff = torch.index_select(p, dim=1, index=idxs)
                wgt_corresp_to_repeat.append(stuff)
                print('stuff.size()', stuff.size())
            wgt_corresp_to_repeat = torch.stack(wgt_corresp_to_repeat, 0)
            print('wgt_corresp_to_repeat.size()', wgt_corresp_to_repeat.size())
            weight = wgt_corresp_to_repeat.sum(dim=0)
            print('weight.size()', weight.size())
            small_state_dict[name] = weight
            



NameError: name 'state_dict' is not defined

In [50]:
for name, p in model.named_parameters():
    i
    print(name, p.size())



save_path = './saved_models_old/cifar10/resnet18bn_ws'+str(ws_amt)+'_ch1'
checkpoint = torch.load(save_path + '/ckpt.pth')

new_state_dict = {}
for k, v in checkpoint['net'].items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)

conv1.weight torch.Size([64, 3, 3, 3])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([64, 64, 3, 3])
layer2.0.bn1.weight torch.Size([64])
layer2.0.bn1.bias torch.Size([64])
layer2.0.conv2.weight torch.Size([64, 128, 3, 3])
layer2.0.bn2.weight torch.Size([64])
layer2.0.bn2.bias torch.Size([64])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight torch.Siz

<All keys matched successfully>

## Weight sharing beats removing filters directly
* Even training both for 350 epochs, plenty of time for

In [11]:
for i in [1, 2, 4, 8, 16]:
    save_path = './saved_models/cifar10/resnet18_ws1_ch'+str(i)
    save_path_ws = './saved_models/cifar10/resnet18_ws'+str(i)+'_ch1'
    checkpoint = torch.load(save_path + '/ckpt.pth')
    acc = checkpoint['acc']
    checkpoint = torch.load(save_path_ws + '/ckpt.pth')
    acc_ws = checkpoint['acc']
    print(f'{i} acc frac: {acc} acc ws: {acc_ws}')

1 acc frac: 95.49 acc ws: 95.49
2 acc frac: 94.79 acc ws: 94.98
4 acc frac: 93.9 acc ws: 94.62
8 acc frac: 92.27 acc ws: 94.02
16 acc frac: 89.51 acc ws: 93.31


In [2]:
imagenet_model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=1, channel_factor=1,
                         num_classes=10, cifar=False)
cifar_model = ResNet18()


imagenet_model_names = []
imagenet_model_sizes = []
for name, param in imagenet_model.named_parameters():
    imagenet_model_names.append(name)
    imagenet_model_sizes.append(param.size())
    
cifar_model_names = []
cifar_model_sizes = []
for name, param in cifar_model.named_parameters():
    cifar_model_names.append(name)
    cifar_model_sizes.append(param.size())

In [5]:
imagenet_model = ResNetLight(BasicBlockLight, [2, 2, 2, 2], ws_factor=1, channel_factor=1,
                         num_classes=10, cifar=True)

imagenet_model.to('cuda')
y = imagenet_model(torch.randn(2,3,32,32).to('cuda'))

0 torch.Size([2, 64, 32, 32])
1 torch.Size([2, 64, 32, 32])
2 torch.Size([2, 128, 16, 16])
3 torch.Size([2, 256, 8, 8])
4 torch.Size([2, 512, 4, 4])
pool torch.Size([2, 512, 1, 1])
